In [1]:
# we'll first collect data approved by FDA
#Data used in this project is taken from this research article-supplimentary information section-"Quantitative structure-activity relationship models for predicting drug-induced liver injury based on FDA-approved drug labelling annotation and using a large collection of drugs"
#"DeepDILI: Deep learning-Powered Drug-Induced Liver Injury Prediction using Model-Level Representation"
#Most of the research papers uses NCTR Liver Cancer Database(NCTRIcdb)


The main aim of this project is to predict the Drug-induced liver injury (DILI) which is a major factor in the development of drugs and the safety of drugs. If the DILI cannot be effectively predicted during the development of the drug, it will cause the drug to be withdrawn from markets.
Physicochemical nature of compounds like lipophilicity has been identified as an important risk factor for DILI when considering together with daily dose. So various physicochemical properties of the molecules generated using padelpy descriptors.
Drug-induced liver injury (DILI) is a major factor in the development of drugs and the safety of drugs. If the DILI cannot be effectively predicted during the development of the drug, it will cause the drug to be withdrawn from markets. Therefore, DILI is crucial at the early stages of drug research. This work presents a 2-class ensemble classifier model for predicting DILI, with 2D molecular descriptors and fingerprints on a dataset of 450 compounds. The purpose of our study is to investigate which are the key molecular fingerprints that may cause DILI risk, and then to obtain a reliable ensemble model to predict DILI risk with these key factors. Experimental results suggested that 8 molecular fingerprints are very critical for predicting DILI, and also obtained the best ratio of molecular fingerprints to molecular descriptors. The result of the 5-fold cross-validation of the ensemble vote classifier method obtain an accuracy of 77.25%, and the accuracy of the test set was 81.67%. This model could be used for drug-induced liver injury prediction.


#References
Minjun Chen, Huixiao Hong, Hong Fang, Reagan Kelly, Guangxu Zhou, Jürgen Borlak, Weida Tong, Quantitative Structure-Activity Relationship Models for Predicting Drug-Induced Liver Injury Based on FDA-Approved Drug Labeling Annotation and Using a Large Collection of Drugs, Toxicological Sciences, Volume 136, Issue 1, November 2013, Pages 242–249, https://doi.org/10.1093/toxsci/kft189


In [71]:
import pandas as pd
import numpy as np

#RDkit library for generation & further processing of physicochemical properties
import rdkit
from rdkit import Chem, DataStructs, RDConfig
from rdkit.Chem import AllChem
from rdkit.Chem import MACCSkeys
from rdkit.ML.Descriptors import MoleculeDescriptors
from rdkit.Chem import Descriptors as des
from rdkit.Chem.Descriptors import qed
from rdkit.Chem import QED
from rdkit import Chem
import rdkit.Chem.AllChem
from rdkit import rdBase
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import Draw
from rdkit.Chem import MolStandardize
from rdkit.Chem.Scaffolds import MurckoScaffold
rdBase.rdkitVersion
%matplotlib inline


from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import cross_val_score




In [3]:
!pip install xlrd

In [4]:
#we can open different sheets present in excel using the sheet_name option like 0,1,2,3 etc
S1=pd.read_excel('/Users/promilasharan/downloads/kft189_Supplementary_Data/toxsci_13_0303_File013.xls', sheet_name=0)

In [5]:
S2=pd.read_excel('/Users/promilasharan/downloads/kft189_Supplementary_Data/toxsci_13_0303_File013.xls', sheet_name=1)

In [6]:
S1.head(5)

,PubChem,CompoundName,Drug label-based annotation
0,3474,glafenine,Most DILI-concern
1,2478,busulfan,Most DILI-concern
2,2520,verapamil,Most DILI-concern
3,2662,celecoxib,Most DILI-concern
4,2898,cyclofenil,Most DILI-concern


In [7]:
S1["Drug label-based annotation"].value_counts()

no DILI-concern      116
Most DILI-concern     81
Name: Drug label-based annotation, dtype: int64

In [8]:
S2.head()

,PubChem,CompoundName,Drug label-based annotation,QSAR Prediction
0,28417,danazol,Most DILI-concern,0
1,65027,tipranavir,Most DILI-concern,1
2,50599,didanosine,Most DILI-concern,1
3,18283,stavudine,Most DILI-concern,1
4,60825,lamivudine,Most DILI-concern,1


In [9]:
S2.shape

(190, 4)

In [10]:
S2["Drug label-based annotation"].value_counts()

Most DILI-concern    95
no DILI-concern      95
Name: Drug label-based annotation, dtype: int64

In [11]:
df1=pd.read_csv('/Users/promilasharan/downloads/S1.csv')

In [12]:
df1.head(5)

,cid,cmpdname,cmpdsynonym,mw,mf,polararea,complexity,xlogp,heavycnt,hbonddonor,...,inchikey,iupacname,meshheadings,annothits,annothitcnt,aids,cidcdate,sidsrcname,depcatg,annotation
0,298,"2,2-dichloro-N-[1,3-dihydroxy-1-(4-nitrophenyl...","2,2-dichloro-N-[1,3-dihydroxy-1-(4-nitrophenyl...",323.13,C11H12Cl2N2O5,115.0,342.0,1.1,20,3,...,WIIZWVCIJKGZOK-UHFFFAOYSA-N,"2,2-dichloro-N-[1,3-dihydroxy-1-(4-nitrophenyl...",NaN,Biological Test Results|Chemical and Physical ...,11,155|157|175|248|256|328|485|631|731|757|758|75...,20050325,001Chemical|AAA Chemistry|abcr GmbH|ABI Chem|A...,Chemical Vendors|Curation Efforts|Governmental...,NaN
1,338,Salicylic acid,salicylic acid|2-Hydroxybenzoic acid|69-72-7|o...,138.12,C7H6O3,57.5,133.0,2.3,10,2,...,YGSDEFSMJLZEOE-UHFFFAOYSA-N,2-hydroxybenzoic acid,Salicylic Acid,Biological Test Results|Biomolecular Interacti...,16,155|157|161|165|167|175|179|248|485|568|583|59...,20040916,001Chemical|3B Scientific (Wuhan) Corp|3WAY PH...,Chemical Vendors|Curation Efforts|Governmental...,NaN
2,938,Nicotinic acid,nicotinic acid|niacin|59-67-6|Pyridine-3-carbo...,123.11,C6H5NO2,50.2,114.0,0.4,9,1,...,PVNIIMVLHYAWGP-UHFFFAOYSA-N,pyridine-3-carboxylic acid,Niacin,Biological Test Results|Biomolecular Interacti...,17,192|248|357|410|411|444|445|446|447|448|450|45...,20040916,001Chemical|3B Scientific (Wuhan) Corp|3WAY PH...,Chemical Vendors|Curation Efforts|Governmental...,"COVID-19, COVID19, Coronavirus, Corona-virus, ..."
3,988,DL-Pantothenic acid,"DL-Pantothenic acid|599-54-2|CHEBI:7916|3-[(2,...",219.23,C9H17NO5,107.0,239.0,-1.1,15,4,...,GHOKWGTUZJEAQD-UHFFFAOYSA-N,"3-[(2,4-dihydroxy-3,3-dimethylbutanoyl)amino]p...",NaN,Biomolecular Interactions and Pathways|Classif...,7,NaN,20040916,001Chemical|3B Scientific (Wuhan) Corp|3WAY PH...,Chemical Vendors|Curation Efforts|Governmental...,NaN
4,1054,Pyridoxine,pyridoxine|65-23-6|vitamin B6|Pyridoxol|Gravid...,169.18,C8H11NO3,73.6,142.0,-0.8,12,3,...,LXNHXLLTXMVWPM-UHFFFAOYSA-N,"4,5-bis(hydroxymethyl)-2-methylpyridin-3-ol",Pyridoxine,Biological Test Results|Biomolecular Interacti...,15,357|410|411|444|445|446|447|448|450|451|526|53...,20040916,001Chemical|3B Scientific (Wuhan) Corp|3WAY PH...,Chemical Vendors|Curation Efforts|Governmental...,"COVID-19, COVID19, Coronavirus, Corona-virus, ..."


In [13]:
df1.columns

Index(['cid', 'cmpdname', 'cmpdsynonym', 'mw', 'mf', 'polararea', 'complexity',
       'xlogp', 'heavycnt', 'hbonddonor', 'hbondacc', 'rotbonds', 'inchi',
       'isosmiles', 'inchikey', 'iupacname', 'meshheadings', 'annothits',
       'annothitcnt', 'aids', 'cidcdate', 'sidsrcname', 'depcatg',
       'annotation'],
      dtype='object')

In [14]:
df2=pd.read_csv('/Users/promilasharan/downloads/S2.csv')

In [15]:
df2.head(5)

,cid,cmpdname,cmpdsynonym,mw,mf,polararea,complexity,xlogp,heavycnt,hbonddonor,...,inchikey,iupacname,meshheadings,annothits,annothitcnt,aids,cidcdate,sidsrcname,depcatg,annotation
0,247,Betaine,betaine|107-43-7|glycine betaine|oxyneurine|ly...,117.15,C5H11NO2,40.1,87.6,0.5,8,0,...,KWIUHFFTVRNATP-UHFFFAOYSA-N,2-(trimethylazaniumyl)acetate,Betaine,Biological Test Results|Biomolecular Interacti...,15,192|248|608|1033|1195|1549|1552|1637|1648|7104...,20040916,001Chemical|3B Scientific (Wuhan) Corp|3WAY PH...,Chemical Vendors|Curation Efforts|Governmental...,NaN
1,453,Hexitol,"Hexitol|hexane-1,2,3,4,5,6-hexaol|hexane-1,2,3...",182.17,C6H14O6,121.0,105.0,-3.1,12,6,...,FBPFZTCFMRRESA-UHFFFAOYSA-N,"hexane-1,2,3,4,5,6-hexol",NaN,Biological Test Results|Classification|Literat...,7,155|157|161|165|167|175|188|200|208|214|220|24...,20050325,001Chemical|3B Scientific (Wuhan) Corp|3WAY PH...,Chemical Vendors|Curation Efforts|Governmental...,NaN
2,679,Dimethyl sulfoxide,dimethyl sulfoxide|DMSO|67-68-5|Methyl sulfoxi...,78.14,C2H6OS,36.3,29.0,-0.6,4,0,...,IAZDPXIOMUYVGZ-UHFFFAOYSA-N,methylsulfinylmethane,Dimethyl Sulfoxide,Biological Test Results|Biomolecular Interacti...,16,179|180|186|192|194|196|206|212|220|222|226|24...,20040916,001Chemical|3B Scientific (Wuhan) Corp|3WAY PH...,Chemical Vendors|Curation Efforts|Governmental...,NaN
3,681,Dopamine,"dopamine|4-(2-Aminoethyl)benzene-1,2-diol|3-Hy...",153.18,C8H11NO2,66.5,119.0,-1.0,11,3,...,VYFYYTLLBUKUHU-UHFFFAOYSA-N,"4-(2-aminoethyl)benzene-1,2-diol",Dopamine,Biological Test Results|Biomolecular Interacti...,16,200|256|357|410|411|444|445|446|447|448|450|45...,20040916,001Chemical|3B Scientific (Wuhan) Corp|3WAY PH...,Chemical Vendors|Curation Efforts|Governmental...,NaN
4,1923,8-Hydroxyquinoline,8-HYDROXYQUINOLINE|quinolin-8-ol|8-quinolinol|...,145.16,C9H7NO,33.1,138.0,2.0,11,1,...,MCJGNVYPOGVAJF-UHFFFAOYSA-N,quinolin-8-ol,Oxyquinoline,Agrochemical Information|Biological Test Resul...,16,1|3|5|7|9|11|13|15|17|19|21|23|25|27|29|31|33|...,20050325,001Chemical|3B Scientific (Wuhan) Corp|3WAY PH...,Chemical Vendors|Curation Efforts|Governmental...,NaN


In [16]:
df1.shape, df2.shape

((193, 24), (184, 24))

In [17]:
S1=S1.rename(columns={"PubChem":"cid"})
#as in df2, df3 we have cid as chemical ids so for further concatenation purposes we must align it with other two dfs

In [18]:
S1.head()

,cid,CompoundName,Drug label-based annotation
0,3474,glafenine,Most DILI-concern
1,2478,busulfan,Most DILI-concern
2,2520,verapamil,Most DILI-concern
3,2662,celecoxib,Most DILI-concern
4,2898,cyclofenil,Most DILI-concern


In [19]:
S2=S2.rename(columns={"PubChem":"cid"})
S2.head(5)

,cid,CompoundName,Drug label-based annotation,QSAR Prediction
0,28417,danazol,Most DILI-concern,0
1,65027,tipranavir,Most DILI-concern,1
2,50599,didanosine,Most DILI-concern,1
3,18283,stavudine,Most DILI-concern,1
4,60825,lamivudine,Most DILI-concern,1


In [20]:
df1=df1[["cid", "isosmiles"]]

In [21]:
df2=df2[["cid", "isosmiles"]]

In [22]:
combined=pd.concat([df1, df2], axis=0)

In [23]:
combined.head(5)

,cid,isosmiles
0,298,C1=CC(=CC=C1C(C(CO)NC(=O)C(Cl)Cl)O)[N+](=O)[O-]
1,338,C1=CC=C(C(=C1)C(=O)O)O
2,938,C1=CC(=CN=C1)C(=O)O
3,988,CC(C)(CO)C(C(=O)NCCC(=O)O)O
4,1054,CC1=NC=C(C(=C1O)CO)CO


In [24]:
combined.shape

(377, 2)

In [25]:
newdata_S1=pd.merge(S1, df1, on ="cid")

In [26]:
newdata_S1.shape

(193, 4)

In [27]:
newdata_S2=pd.merge(S2, df2, on ="cid")

In [28]:
newdata_S2.shape

(184, 5)

In [29]:
newdata_S2.columns

Index(['cid', 'CompoundName', 'Drug label-based annotation', 'QSAR Prediction',
       'isosmiles'],
      dtype='object')

In [30]:
newdata_S1["Drug label-based annotation"].value_counts()

no DILI-concern      116
Most DILI-concern     77
Name: Drug label-based annotation, dtype: int64

In [31]:
newdata_S2["Drug label-based annotation"].value_counts()

no DILI-concern      93
Most DILI-concern    91
Name: Drug label-based annotation, dtype: int64

In [32]:
combined=pd.concat([newdata_S1,newdata_S2], axis=0)

In [33]:
combined.head()

,cid,CompoundName,Drug label-based annotation,isosmiles,QSAR Prediction
0,3474,glafenine,Most DILI-concern,C1=CC=C(C(=C1)C(=O)OCC(CO)O)NC2=C3C=CC(=CC3=NC...,NaN
1,2478,busulfan,Most DILI-concern,CS(=O)(=O)OCCCCOS(=O)(=O)C,NaN
2,2520,verapamil,Most DILI-concern,CC(C)C(CCCN(C)CCC1=CC(=C(C=C1)OC)OC)(C#N)C2=CC...,NaN
3,2662,celecoxib,Most DILI-concern,CC1=CC=C(C=C1)C2=CC(=NN2C3=CC=C(C=C3)S(=O)(=O)...,NaN
4,2898,cyclofenil,Most DILI-concern,CC(=O)OC1=CC=C(C=C1)C(=C2CCCCC2)C3=CC=C(C=C3)O...,NaN


In [34]:
combined.shape

(377, 5)

In [35]:
combined.to_csv("DILI-dataset.csv", index=None)

In [36]:
DILI_active=combined[combined["Drug label-based annotation"].str.contains("Most DILI-concern")]

In [37]:
DILI_active.shape

(168, 5)

In [38]:
DILI_inactive=combined[combined["Drug label-based annotation"].str.contains("no DILI-concern")]

In [39]:
DILI_inactive.shape

(209, 5)

In [40]:
DILI_active["label"]=1
DILI_inactive["label"]=0

/Users/promilasharan/opt/anaconda3/envs/uresearcher/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/Users/promilasharan/opt/anaconda3/envs/uresearcher/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [41]:
DILI=pd.concat([DILI_active,DILI_inactive], axis=0)

In [42]:
DILI[["isosmiles", "label"]].to_csv("DILI_labelled.smi", header=None, sep="\t", index=None)

# as we are taking into consideration the liver toxicity of drugs so we can generate descriptors corresponding to physicochemical properties

In [43]:
#Morgan fingerprints
smi=[Chem.MolFromSmiles (mol) for mol in DILI["isosmiles"].iloc[0:] if mol is not None]

In [44]:
smi

In [45]:
len(smi)

377

In [46]:
train_DILI_fp = [AllChem.GetMorganFingerprintAsBitVect(mol,2, nBits=1024) for mol in smi if mol is not None]

In [47]:
fp_np=np.asarray(train_DILI_fp, dtype=np.int32)

In [48]:
fp_np

array([[0, 1, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 1, 0]], dtype=int32)

In [49]:
DILI.head()

,cid,CompoundName,Drug label-based annotation,isosmiles,QSAR Prediction,label
0,3474,glafenine,Most DILI-concern,C1=CC=C(C(=C1)C(=O)OCC(CO)O)NC2=C3C=CC(=CC3=NC...,NaN,1
1,2478,busulfan,Most DILI-concern,CS(=O)(=O)OCCCCOS(=O)(=O)C,NaN,1
2,2520,verapamil,Most DILI-concern,CC(C)C(CCCN(C)CCC1=CC(=C(C=C1)OC)OC)(C#N)C2=CC...,NaN,1
3,2662,celecoxib,Most DILI-concern,CC1=CC=C(C=C1)C2=CC(=NN2C3=CC=C(C=C3)S(=O)(=O)...,NaN,1
4,2898,cyclofenil,Most DILI-concern,CC(=O)OC1=CC=C(C=C1)C(=C2CCCCC2)C3=CC=C(C=C3)O...,NaN,1


In [50]:
#sometimes we can't go for this method as we used the command that generate fp if mol is not none. so if there is some molecules too large or are none then fp are generated w.r.t that molecules.
#so if we use the label of original dataset against the fp then it will not yield accurate results.
X_train, X_test, y_train, y_test = train_test_split(fp_np, DILI["label"].ravel(), random_state=42, test_size=0.2, shuffle=True, stratify=DILI["label"])

In [51]:
#alternative method for this is-to use the supplier
supplier=Chem.SmilesMolSupplier("DILI_labelled.smi", delimiter="\t", titleLine=False)

In [52]:
#so if there is smiles missing, it will not bring the corresponding label as well in final dataset
new_fp=[]
labels=[]
for i, mol in enumerate(supplier):
    if mol is not None:
        new_fp.append(rdkit.Chem.AllChem.GetMorganFingerprintAsBitVect(mol,2,nBits=1024))
        labels.append(mol.GetProp('_Name'))

In [53]:
len(labels)

377

In [54]:
len(new_fp)

377

In [55]:
fp_np=np.asarray(new_fp, dtype=np.int32)
ids=np.asarray(labels, int).reshape(-1,1)

In [56]:
X_train, X_test, y_train, y_test = train_test_split(fp_np, ids, random_state=42, test_size=0.2, shuffle=True)

In [57]:
X_train.shape

(301, 1024)

In [58]:
X_test.shape

(76, 1024)

In [59]:
#Now we will build random forest classifier
rf=RandomForestClassifier(n_estimators=1000, random_state=42)
rf.fit(X_train, y_train.ravel())

RandomForestClassifier(n_estimators=1000, random_state=42)

In [60]:
predicted=rf.predict(X_test)

In [61]:
from pycm import *
cm = ConfusionMatrix(y_test.reshape(-1), predicted)
print(cm)
#check for MCC (its used in drug discovery), AUC

Predict  0        1        
Actual
0        32       4        

1        18       22       





Overall Statistics : 

95% CI                                                            (0.60856,0.81249)
ACC Macro                                                         0.71053
ARI                                                               0.16708
AUNP                                                              0.71944
AUNU                                                              0.71944
Bangdiwala B                                                      0.53099
Bennett S                                                         0.42105
CBA                                                               0.595
CSI                                                               0.46252
Chi-Squared                                                       16.21593
Chi-Squared DF                                                    1
Conditional Entropy                                            

In [62]:
from sklearn.svm import SVC
svm = SVC()

In [63]:
svm.fit(X_train, y_train.ravel())

SVC()

In [64]:
pd.set_option('display.max_rows',1000)
predicted_svm=svm.predict(X_test)
cm_svc=ConfusionMatrix(y_test.reshape(-1), predicted_svm)
print(cm_svc)

Predict  0        1        
Actual
0        34       2        

1        19       21       





Overall Statistics : 

95% CI                                                            (0.62315,0.82422)
ACC Macro                                                         0.72368
ARI                                                               0.19071
AUNP                                                              0.73472
AUNU                                                              0.73472
Bangdiwala B                                                      0.56471
Bennett S                                                         0.44737
CBA                                                               0.58325
CSI                                                               0.512
Chi-Squared                                                       19.78522
Chi-Squared DF                                                    1
Conditional Entropy                                            

In [65]:
#we can do hyperparameter optimization

In [66]:
#from padelpy import padelpy descriptors
#or we can genearte using smiles

In [67]:
#we can save a model in pickle
import pickle
with open("DILI_rf_model.pkl", "wb") as f:#here w in wb stands for write
    clf=pickle.dump(rf, f)

In [68]:
#if we want to test the new dataset using the rf classifier, we need to follow the following instructions
#here w in wb stands for write
import pickle
with open("DILI_rf_model.pkl", "rb") as f:
    clf=pickle.load(f)